In [1]:
import torch
import torch.nn as nn
from torch.nn import functional as F

In [2]:
batch_size = 16 
block_size = 32 
max_iters = 5000
eval_interval = 100
learning_rate = 1e-3
device = 'cuda' if torch.cuda.is_available() else 'cpu'
eval_iters = 200
n_embd = 64
n_head = 4
n_layer = 4
dropout = 0.0

In [3]:
torch.manual_seed(1337)

In [4]:
with open('thai green curry.txt', 'r', encoding='utf-8') as f:
    text = f.read()

In [5]:
chars = sorted(list(set(text)))
vocab_size = len(chars)

print(''.join(chars))
print(vocab_size)


 ()-.123456789กขคงจฉชซณดตถทธนบปผพฟภมยรลวษสหอะัาำิีึืุูเแโใไๆ็่้์
65


In [6]:
stoi = { ch:i for i,ch in enumerate(chars) }
itos = { i:ch for i,ch in enumerate(chars) }
encode = lambda s: [stoi[c] for c in s]
pre_decode = lambda l: [itos[i] for i in l]
decode = lambda l: ''.join([itos[i] for i in l])

print(stoi)
print('---')
print(itos)
print('---')
print(encode('แกง'))
print(pre_decode(encode('แกง')))
print(decode(encode('แกง')))

{'\n': 0, ' ': 1, '(': 2, ')': 3, '-': 4, '.': 5, '1': 6, '2': 7, '3': 8, '4': 9, '5': 10, '6': 11, '7': 12, '8': 13, '9': 14, 'ก': 15, 'ข': 16, 'ค': 17, 'ง': 18, 'จ': 19, 'ฉ': 20, 'ช': 21, 'ซ': 22, 'ณ': 23, 'ด': 24, 'ต': 25, 'ถ': 26, 'ท': 27, 'ธ': 28, 'น': 29, 'บ': 30, 'ป': 31, 'ผ': 32, 'พ': 33, 'ฟ': 34, 'ภ': 35, 'ม': 36, 'ย': 37, 'ร': 38, 'ล': 39, 'ว': 40, 'ษ': 41, 'ส': 42, 'ห': 43, 'อ': 44, 'ะ': 45, 'ั': 46, 'า': 47, 'ำ': 48, 'ิ': 49, 'ี': 50, 'ึ': 51, 'ื': 52, 'ุ': 53, 'ู': 54, 'เ': 55, 'แ': 56, 'โ': 57, 'ใ': 58, 'ไ': 59, 'ๆ': 60, '็': 61, '่': 62, '้': 63, '์': 64}
---
{0: '\n', 1: ' ', 2: '(', 3: ')', 4: '-', 5: '.', 6: '1', 7: '2', 8: '3', 9: '4', 10: '5', 11: '6', 12: '7', 13: '8', 14: '9', 15: 'ก', 16: 'ข', 17: 'ค', 18: 'ง', 19: 'จ', 20: 'ฉ', 21: 'ช', 22: 'ซ', 23: 'ณ', 24: 'ด', 25: 'ต', 26: 'ถ', 27: 'ท', 28: 'ธ', 29: 'น', 30: 'บ', 31: 'ป', 32: 'ผ', 33: 'พ', 34: 'ฟ', 35: 'ภ', 36: 'ม', 37: 'ย', 38: 'ร', 39: 'ล', 40: 'ว', 41: 'ษ', 42: 'ส', 43: 'ห', 44: 'อ', 45: 'ะ', 46: 'ั', 47: 

In [7]:
data = torch.tensor(encode(text), dtype=torch.long)
n = int(0.9*len(data))
train_data = data[:n]
val_data = data[n:]

In [8]:
def get_batch(split):
    
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y

@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

class Head(nn.Module):

    def __init__(self, head_size):
        super().__init__()
        self.key = nn.Linear(n_embd, head_size, bias=False)
        self.query = nn.Linear(n_embd, head_size, bias=False)
        self.value = nn.Linear(n_embd, head_size, bias=False)
        self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))

        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        B,T,C = x.shape
        k = self.key(x)   
        q = self.query(x) 
        
        wei = q @ k.transpose(-2,-1) * C**-0.5 
        wei = wei.masked_fill(self.tril[:T, :T] == 0, float('-inf'))
        wei = F.softmax(wei, dim=-1) 
        wei = self.dropout(wei)
        
        v = self.value(x)
        out = wei @ v 
        return out

class MultiHeadAttention(nn.Module):

    def __init__(self, num_heads, head_size):
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
        self.proj = nn.Linear(n_embd, n_embd)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        out = torch.cat([h(x) for h in self.heads], dim=-1)
        out = self.dropout(self.proj(out))
        return out

class FeedFoward(nn.Module):

    def __init__(self, n_embd):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embd, 4 * n_embd),
            nn.ReLU(),
            nn.Linear(4 * n_embd, n_embd),
            nn.Dropout(dropout),
        )

    def forward(self, x):
        return self.net(x)

class Block(nn.Module):

    def __init__(self, n_embd, n_head):
        
        super().__init__()
        head_size = n_embd // n_head
        self.sa = MultiHeadAttention(n_head, head_size)
        self.ffwd = FeedFoward(n_embd)
        self.ln1 = nn.LayerNorm(n_embd)
        self.ln2 = nn.LayerNorm(n_embd)

    def forward(self, x):
        x = x + self.sa(self.ln1(x))
        x = x + self.ffwd(self.ln2(x))
        return x


class BigramLanguageModel(nn.Module):

    def __init__(self):
        super().__init__()
        
        self.token_embedding_table = nn.Embedding(vocab_size, n_embd)
        self.position_embedding_table = nn.Embedding(block_size, n_embd)
        self.blocks = nn.Sequential(*[Block(n_embd, n_head=n_head) for _ in range(n_layer)])
        self.ln_f = nn.LayerNorm(n_embd)
        self.lm_head = nn.Linear(n_embd, vocab_size)

    def forward(self, idx, targets=None):
        B, T = idx.shape

    
        tok_emb = self.token_embedding_table(idx) 
        pos_emb = self.position_embedding_table(torch.arange(T, device=device)) 
        x = tok_emb + pos_emb 
        x = self.blocks(x) 
        x = self.ln_f(x)
        logits = self.lm_head(x) 

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, idx, max_new_tokens):
        
        for _ in range(max_new_tokens):
          
            idx_cond = idx[:, -block_size:]
   
            logits, loss = self(idx_cond)
           
            logits = logits[:, -1, :] 
         
            probs = F.softmax(logits, dim=-1) 
           
            idx_next = torch.multinomial(probs, num_samples=1)
         
            idx = torch.cat((idx, idx_next), dim=1) 
        return idx

model = BigramLanguageModel()
m = model.to(device)

print(sum(p.numel() for p in m.parameters())/1e6, 'M parameters')

0.209729 M parameters


In [9]:
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

for iter in range(max_iters):

  
    if iter % eval_interval == 0 or iter == max_iters - 1:
        losses = estimate_loss()
        print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")


    xb, yb = get_batch('train')


    logits, loss = model(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()


context = torch.zeros((1, 1), dtype=torch.long, device=device)
print(decode(m.generate(context, max_new_tokens=2000)[0].tolist()))

step 0: train loss 4.3463, val loss 4.3735
step 100: train loss 2.4102, val loss 3.1430
step 200: train loss 1.7384, val loss 3.0545
step 300: train loss 1.0356, val loss 3.1172
step 400: train loss 0.5564, val loss 3.4640
step 500: train loss 0.3422, val loss 3.6903
step 600: train loss 0.2747, val loss 3.9265
step 700: train loss 0.2379, val loss 4.1296
step 800: train loss 0.2095, val loss 4.2324
step 900: train loss 0.2006, val loss 4.3205
step 1000: train loss 0.1913, val loss 4.4204
step 1100: train loss 0.1806, val loss 4.5349
step 1200: train loss 0.1709, val loss 4.6064
step 1300: train loss 0.1737, val loss 4.6378
step 1400: train loss 0.1714, val loss 4.7553
step 1500: train loss 0.1665, val loss 4.6400
step 1600: train loss 0.1668, val loss 4.7125
step 1700: train loss 0.1672, val loss 4.7119
step 1800: train loss 0.1620, val loss 4.7055
step 1900: train loss 0.1530, val loss 4.8309
step 2000: train loss 0.1582, val loss 4.7945
step 2100: train loss 0.1552, val loss 4.8778
